In [1]:
library("DBI")
library("dbplyr")
library("dplyr")

con <- DBI::dbConnect(
    RPostgres::Postgres(),
    host = "localhost",
    port = 5432,
    dbname = "knock100",
    user = "guest",
    password = "guest"
)

customer_tbl <- dplyr::tbl(con, "customer")
category_tbl <- dplyr::tbl(con, "category")
product_tbl <- dplyr::tbl(con, "product")
receipt_tbl <- dplyr::tbl(con, "receipt")
store_tbl <- dplyr::tbl(con, "store")
geo_tbl <- dplyr::tbl(con, 'geocode')
DBI::dbListTables(con)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:dbplyr’:

    ident, sql


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


[1] "customer" "category" "geocode"  "product"  "receipt"  "store"

### R-080:
商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [ ]:
# missing data
product_tbl %>% filter(if_any(everything(), ~ is.na(.))) %>% head

In [ ]:
product_tbl %>% filter(if_all(everything(), ~ !is.na(.))) %>% head

In [ ]:
# complete.cases does not works on tbl
product_tbl %>% collect() %>%
filter(complete.cases(.)) %>% nrow()

### R-081:
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
library(tidymodels)

product_df <- data.frame(con, 'product')
product_tbl %>%
recipe() %>%
step_impute_mean(unit_price, unit_cost) %>%
prep() %>%
juice() %>%
head
# filter(unit_cost %>% is.na)

In [ ]:
product_tbl %>% 
mutate(across(starts_with("unit_"), ~ coalesce(.x, mean(.x) %>% round())))

In [ ]:
rows_to_impute <- c("unit_price", "unit_cost")
product_tbl %>%
mutate(across(rows_to_impute, ~ coalesce(.x, mean(.x) %>% round())))

### R-082:
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

### R-083:
単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
product_tbl %>% 
collect() %>%
group_by(category_small_cd) %>%
group_modify(~ mutate(.x, 
	across(
		.cols = starts_with("unit_"),
		.fns = ~ coalesce(.x, median(.x, na.rm = TRUE) %>% round())
))) %>% head

In [109]:
rows_to_impute <- c("unit_price", "unit_cost")
product_tbl %>%
group_by(category_small_cd) %>%
summarise(
	across(
		.cols = all_of(rows_to_impute),
		.fns = ~ median(.x) %>% round(),
		.names = '{.col}_med'
		) 
	) %>%
inner_join(product_tbl %>% filter(if_any(everything(), ~ is.na(.))), by = 'category_small_cd') %>%
collect() %>% 
group_by(category_small_cd) %>% 
group_modify(
	~ mutate(
		.x,
		across(
			.cols = all_of(rows_to_impute),
			.fns = ~ coalesce(.x, cur_data() %>% summarise(across(stringr::str_c(cur_column(),'_med') ,first)) %>% collect() %>% as.numeric()) 
			))
		) %>% 
select(- ends_with('_med')) %>% 
# copy_to method temporarily saves table in server, and then merge tables
copy_to(con, ., overwrite = TRUE) %>% dplyr::union(product_tbl %>% filter(if_all(everything(), ~ !is.na(.)))) %>%
# make sure that the merged table has no missing cells
summarise(across(.cols = everything(), .fns = ~ if_else(is.na(.x), 1, 0) %>% sum(), .names = '{.col}_NA'))

# Source:   SQL [1 x 6]
# Database: postgres  [guest@localhost:5432/knock100]
  category_small_… product_cd_NA category_major_… category_medium… unit_price_NA
             <dbl>         <dbl>            <dbl>            <dbl>         <dbl>
1                0             0                0                0             0
# … with 1 more variable: unit_cost_NA <dbl>

In [ ]:
pr %>%
group_by(category_small_cd) %>%
summarise(
		price_median = unit_price %>% median(na.rm = TRUE) %>% round(),
		cost_median = unit_cost %>% median(na.rm = TRUE) %>% round()
		) %>%
inner_join(pr, by='category_small_cd') %>%
mutate(
	unit_price = ifelse(unit_price %>% is.na, price_median, unit_price),
	unit_cost = ifelse(unit_cost %>% is.na, cost_median, unit_cost)
)

### R-084:
顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [ ]:
receipt_tbl %>% 
mutate(year = sales_ymd %>% as.character() %>% substr(1,4)) %>%
group_by(customer_id) %>%
summarise(
	amount_all = sum(amount, na.rm=TRUE),
	amount_2019 = sum(if_else(year=='2019', coalesce(amount,0), 0), na.rm=TRUE)
)  %>%
 right_join(customer_tbl %>% select(customer_id), by = 'customer_id') %>%
 mutate(across(
 	.cols = starts_with('amount'),
 	.fns = ~ coalesce(.x, .0)
 )) %>% # the first of asked data
 filter(amount_2019>0) %>%
 # the second one
 mutate(ratio_2019 = amount_2019/amount_all) %>% head

In [ ]:
library(lubridate)
cu = tbl(con, 'customer')

re %>%
data.frame %>%
mutate(year = sales_ymd %>% ymd %>% year) %>%
group_by(customer_id, year) %>%
summarize(
	amount_all = sum(amount, na.rm = TRUE),
	amount_2019 = sum(ifelse(year == 2019,amount,0 ), na.rm = TRUE)
) %>%
right_join(cu %>% data.frame, by = 'customer_id') %>%
replace_na(list(amount_all = 0, amount_2019 = 0)) %>%
filter(amount_2019 > 0) %>% head(10)

### R-085:
顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

### R-086:
085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

### R-087:
顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [2]:
customer_tbl_u2 <- receipt_tbl %>%
group_by(customer_id) %>%
summarise(amount = amount %>% sum(na.rm = TRUE)) %>%
right_join(customer_tbl, by = 'customer_id') %>%
group_by(customer_name, postal_cd) %>%
mutate(
	amount = amount %>% coalesce(0),
	multi = n(),
	use_this = (multi==1)
				| ((amount==max(amount) & amount != min(amount)))
				| ( amount==max(amount) & (customer_id == min(customer_id)))
	) %>% 
filter(use_this) %>% 
ungroup() %>% 
select(colnames(customer_tbl))
# show 
customer_tbl_u2 %>% summarise(n_unique_rows = n())

Warning message:
“Missing values are always removed in SQL aggregation functions.
Use `na.rm = TRUE` to silence this warning
This warning is displayed once every 8 hours.”


# Source:   SQL [1 x 1]
# Database: postgres  [guest@localhost:5432/knock100]
  n_unique_rows
        <int64>
1         21941

In [32]:
customer_tbl_u <- customer_tbl %>%
left_join(receipt_tbl %>% select(customer_id, amount), by = 'customer_id') %>%
group_by(across(all_of(customer_tbl %>% colnames()))) %>%
summarise(sales_amount = amount %>% sum() %>% coalesce(0), .groups = 'drop') %>%
group_by(customer_name, postal_cd) %>%
window_order(desc(sales_amount), customer_id) %>% 
distinct(customer_name, postal_cd, .keep_all = TRUE) %>%
ungroup()

customer_tbl_u %>% summarise(n())

# Source:     SQL [1 x 1]
# Database:   postgres  [guest@localhost:5432/knock100]
# Ordered by: desc(sales_amount), customer_id
    `n()`
  <int64>
1   21941

In [35]:
# compare with official answer
df_receipt <- receipt_tbl %>% collect() %>% data.frame()
df_customer <- customer_tbl %>% collect() %>% data.frame()

df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

df_customer_u <- left_join(df_customer, df_sales_amount, by = "customer_id") %>%
    mutate(sum_amount = ifelse(is.na(sum_amount), 0, sum_amount)) %>%
    arrange(desc(sum_amount), customer_id) %>%
    distinct(customer_name, postal_cd, .keep_all = TRUE)

col_names_check <- df_customer %>% colnames()

rows_diff <- setdiff(df_customer_u %>% select(all_of(col_names_check)) , customer_tbl_u %>% select(all_of(col_names_check)) %>% collect() %>% as.data.frame()) 
names_diff <- rows_diff[,'customer_name']
rows_diff

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
<chr>,<chr>,<int>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


In [36]:
# compare diff set 
customer_tbl_u %>% filter(customer_name %in% names_diff) %>% collect()
df_customer_u %>% filter(customer_name %in% names_diff)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,sales_amount
<chr>,<chr>,<int>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,sum_amount
<chr>,<chr>,<int>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>


### R-088:
087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
- 重複していない顧客：顧客ID（customer_id）を設定
- 重複している顧客：前設問で抽出したレコードの顧客IDを設定

顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

## R-089:
売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [82]:
customer_with_sales_history <- receipt_tbl %>%
group_by(customer_id) %>%
summarise(sales_amount = amount %>% sum(na.rm = TRUE), .groups = 'drop') %>% 
filter(sales_amount > 0)

count_rows <- function(table) table %>% summarise(n()) %>% collect() %>% data.frame() %>% .[1,1] %>% as.double()

N <- customer_with_sales_history %>% count_rows()

data_test <- customer_with_sales_history %>% slice_sample(n = 0.2*N, replace = FALSE)
data_train <- dplyr::setdiff(customer_with_sales_history, data_test)

"size of N:" %>% paste(N)
"ratio of training data:" %>% paste(count_rows(data_train) / N )
"ratio of test data:" %>% paste(count_rows(data_test) / N)

ERROR: Error in customer_by_sales %>% filter(sales_amount > 0) <- receipt_tbl %>% : object 'customer_by_sales' not found


### R-090:
レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

## R-091:
顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [100]:
customer_sales_amount <- customer_tbl %>%
left_join(receipt_tbl %>% select(customer_id, amount), by = 'customer_id') %>%
group_by(customer_id) %>%
summarise(amount = amount %>% sum(na.rm = TRUE) %>% coalesce(0), .groups = 'drop') %>%
mutate(purchase = if_else(amount>0,1,0))

customer_with_sales_history <- customer_sales_amount %>% filter(amount>0)

# size of down sampling
size <- count_rows(customer_sales_amount) %>% min(customer_with_sales_history %>% count_rows())

# union each of sampled group
down_samples_tbl <- customer_sales_amount %>% filter(amount==0) %>% slice_sample(n = size) %>% dplyr::union(customer_with_sales_history %>% slice_sample(n = size))

down_samples_tbl %>% group_by(purchase) %>%
summarise(n = n())

# Source:   SQL [2 x 2]
# Database: postgres  [guest@localhost:5432/knock100]
  purchase       n
     <dbl> <int64>
1        1    8306
2        0    8306

## R-092:
顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

## R-093:
 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

## R-094:
093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。

ファイル形式	ヘッダ有無	文字エンコーディング
CSV（カンマ区切り）	有り	UTF-8

## R-095:
093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。

ファイル形式	ヘッダ有無	文字エンコーディング
CSV（カンマ区切り）	有り	CP932

## R-096:
093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。

ファイル形式	ヘッダ有無	文字エンコーディング
CSV（カンマ区切り）	無し	UTF-8